Capstone project on Data Science  
IBM Professional Certificate

0. Installing all required packages

In [4]:
#Installing folium, geocoder amd geopy
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [12]:
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

In [5]:
import pandas as pd
import numpy as np
import json
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

1. Data uploading and processing

In [6]:
# Downloading data and showing first 5 rows
path = 'table-1.csv'
df = pd.read_csv(path)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
# Processing of data
# First of all, we remove all rows where Borough isn't assigned
df.drop(df[df['Borough'] == 'Not assigned'].index,axis=0,inplace=True)
# If Borough is shown, but neighbourhood is unknown, we will consider that Borough = Neighbourhood
df.replace(df[df['Neighbourhood'] == 'Not assigned'].index,df['Borough'][df[df['Neighbourhood'] == 'Not assigned'].index])
# If more than one neighborhood exist in postcode area, we will combine rows
df_new = df.groupby(['Postcode'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
#Remove repeating words in Borough column
newbor = [] #Create a list with clear names
for val in df_new['Borough']:
    val = val.split(' ')
    newbor.append(val[0])
df_new['Borough'] = newbor #Append it instead of old ones
print(df_new.shape)
df_new

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge Malvern
1,M1C,Scarborough,Highland Creek Rouge Hill Port Union
2,M1E,Scarborough,Guildwood Morningside West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village Martin Grove Gardens Richvie...
101,M9V,Etobicoke,Albion Gardens Beaumond Heights Humbergate Jam...


In [8]:
##For some reason this code doesn't work so we just download dataset
#import geocoder # import geocoder
#postal_codes = df_new['Postcode']
#lats = []
#longs = []
#for postal_code in postal_codes:
#    # initialize your variable to None
#    lat_lng_coords = None

    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        e = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = e.latlng
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    lats.append(latitude)
#    longs.append(longitude)

In [9]:
path = 'http://cocl.us/Geospatial_data'
geospatial = pd.read_csv(path)

In [10]:
geospatial.head()
geospatial.shape

(103, 3)

In [11]:
df_new['Latitude'] = geospatial['Latitude']
df_new['Longitude'] = geospatial['Longitude']
df_new

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge Malvern,43.806686,-79.194353
1,M1C,Scarborough,Highland Creek Rouge Hill Port Union,43.784535,-79.160497
2,M1E,Scarborough,Guildwood Morningside West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village Martin Grove Gardens Richvie...,43.688905,-79.554724
101,M9V,Etobicoke,Albion Gardens Beaumond Heights Humbergate Jam...,43.739416,-79.588437


2. Visualisation of Neighbourhoods

In [13]:
# We find the coordinates of Toronto on map
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# We create Folium map and center it around Toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
# Then we add points corresponding to neighbourhoods
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffd800',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

3. Exploration of neighborhoods

In [17]:
# Entering Foursquare creditinals
CLIENT_ID = 'PY1W0T2M3B244JAWRHE414OMP3J2HJPJFNOAPRICZY0B33FG'
CLIENT_SECRET = 'ZS25SVHPO00IPA51SSN1ZOAT3YTDX1HRER5LM1H30Y4Y1A52'
VERSION = '20190909' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PY1W0T2M3B244JAWRHE414OMP3J2HJPJFNOAPRICZY0B33FG
CLIENT_SECRET:ZS25SVHPO00IPA51SSN1ZOAT3YTDX1HRER5LM1H30Y4Y1A52


In [ ]:
loc[0, 'Neighborhood']